## Gold Layer: Analytics-Ready Tables
- Fact tables
- Dimension tables
- Aggregations

#### Defining 3 use cases for business reporting

Each of these tables is designed to give aggregated metrics on average playtime, reviews, and total sales:
    
i.) Category Level - Useful for publishers who want to see what categories (e.g., MMO, VR Supported, Multiplayer, etc.) are performing well, so they can focus on them when prioritizing future games.

ii.) Genre level - This aggregated table functions similarly to the category level, but functions at the genre level (e.g., Horror, Fantasy, Sports), which is also useful for prioritizing future game development.

iii.) Publisher Level - This table allows game publishers to see how their own playtime, review, and sales stats compare to their competitors. It helps answer key questions such as "Are we having a poor performing year, or is this an industry trend that our competitors are facing as well?"

#### Design and populate the Gold layer tables to meet those use cases


In [ ]:
CREATE OR REPLACE TABLE GOLD.CATEGORY_METRICS AS

SELECT 
    CATEGORY,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_CAT_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_CATEGORY b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_CATEGORY c ON b.CATEGORY_KEY = c.CATEGORY_KEY

GROUP BY CATEGORY;

In [ ]:
CREATE OR REPLACE TABLE GOLD.GENRE_METRICS AS 

SELECT 
    GENRE,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_GENRE_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_GENRE b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_GENRE dg ON b.GENRE_KEY = dg.GENRE_KEY

GROUP BY GENRE;

In [ ]:
CREATE OR REPLACE TABLE GOLD.PUBLISHER_METRICS AS

SELECT 
    PUBLISHER,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_PUBLISHER_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_PUBLISHER p ON f.PUBLISHER_KEY = p.PUBLISHER_KEY

GROUP BY PUBLISHER;

In [ ]:
SELECT * FROM GOLD.GENRE_METRICS;